In [45]:
import boto3
import json
import random
import string

In [46]:
def createStudentRecord():
    
    student_id = random.randint(1,100)
    student_name = "".join(random.choices(string.ascii_letters,k=5))
    student_marks = random.randint(0,100)
    return {"student_id":student_id, "student_name":student_name, "student_marks":student_marks}

student_details = createStudentRecord()

fileObject = open("./StudentDetails.txt","w")
json.dump(student_details,fileObject)
fileObject.close()

In [47]:
## S3 BOTO3 API
s3_client = boto3.client('s3')

In [48]:
def create_bucket(bucketName):
    return s3_client.create_bucket(Bucket = bucketName)

bucket_name = "studentmanagementbucket"+ str(random.randint(10,18) * random.randint(100,999))
bucketCreation_Response = create_bucket(bucket_name)

In [49]:
lambda_client = boto3.client('lambda')

In [50]:
lambda_function_arn = lambda_client.get_function( FunctionName = "ProcessStudentDetailsFunction")["Configuration"]["FunctionArn"]

In [51]:
lambda_configuration = {
    "LambdaFunctionConfigurations" :[
        {
            "Id": "EventLambda",
            "LambdaFunctionArn" : lambda_function_arn,
            "Events" : ['s3:ObjectCreated:*']
        }
    ]
}

In [52]:
bucket_arn = f'arn:aws:s3:::{bucket_name}'
lamda_permission_response = lambda_client.add_permission(
    FunctionName = "ProcessStudentDetailsFunction",
    StatementId = str(random.randint(1,1000)),
    Action = "lambda:InvokeFunction",
    Principal = "s3.amazonaws.com",
    SourceArn = bucket_arn
)

In [53]:
if bucketCreation_Response["ResponseMetadata"]["HTTPStatusCode"] == 200 and lamda_permission_response["ResponseMetadata"]["HTTPStatusCode"] == 201:
    s3_client.put_bucket_notification_configuration(
    Bucket=bucket_name,
    NotificationConfiguration=lambda_configuration
    )
    s3_client.upload_file('./StudentDetails.txt',bucket_name,"StudentDetails.txt")
else:
    print(bucketCreation_Response,lamda_permission_response)